In [ ]:
!pip install datasets evaluate
!pip install transformers==4.28.0

In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining,BertModel

# tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
# model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-small-uncased')
model = BertModel.from_pretrained('nlpaueb/legal-bert-small-uncased')


In [21]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 512, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=512, out_features=512, bias=True)
            (key): Linear(in_features=512, out_features=512, bias=True)
            (value): Linear(in_features=512, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=512, out_features=512, bias=True)
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [22]:
inputs = tokenizer("Hello, my dog is cute", return_tensors='pt')
outputs = model(**inputs, output_hidden_states=True)
last_hidden_states = outputs.hidden_states[-1]

In [60]:
outputs.last_hidden_state[:,0,:]

tensor([[ 8.9154e-02,  7.5592e-01, -3.0122e-01, -1.9434e-01,  1.2388e+00,
         -6.7537e-01, -7.7964e-01, -3.2386e-01,  6.8272e-02,  7.4917e-01,
         -4.9709e-01,  9.9172e-01,  8.9748e-01,  1.8401e-01,  1.3438e+00,
          1.4325e+00,  3.1874e-01,  1.2013e-01,  3.7643e-01,  1.9449e-01,
          3.6766e-01,  1.1599e-01, -2.8162e-01, -1.2121e-01, -2.8447e-01,
         -6.3376e-01,  1.1540e+00, -1.5454e-01,  6.8308e-01, -8.5695e-01,
          1.4745e+00, -7.3525e-01, -8.5454e-01,  1.2812e-01,  2.1665e+00,
          1.0175e+00,  6.1710e-01,  8.7967e-01,  1.1696e+00,  2.6238e-01,
          7.3016e-01, -5.3249e-02,  1.9482e-01, -8.4176e-01,  7.5167e-02,
          4.9097e-01,  1.1278e+00, -1.5101e-02, -6.1247e-01,  8.2152e-01,
          9.2536e-02, -6.0423e-01, -6.6079e-01,  7.0307e-02, -4.9960e-01,
         -1.5762e-01,  4.7191e-01,  7.8625e-01, -3.0893e-01, -7.9057e-01,
         -3.7742e-01,  1.0000e+00,  2.3580e+00, -1.5727e-01,  9.9970e-02,
          1.1013e+00,  1.6277e-01, -2.

In [43]:
outputs.hidden_states[-1][0]

tensor([[ 0.0892,  0.7559, -0.3012,  ..., -0.5072,  0.2997, -4.0243],
        [-0.2006, -0.3864, -0.4511,  ..., -1.2621, -1.2866,  1.2020],
        [-0.1936, -1.4339, -1.0280,  ...,  0.1975, -1.7349,  1.3770],
        ...,
        [ 0.5568,  0.3182,  0.4068,  ..., -1.0688, -0.6562,  0.2717],
        [ 0.3446, -0.0966,  1.0200,  ..., -0.8399, -0.4163,  0.3802],
        [ 0.6180,  0.3152, -0.6200,  ..., -0.4392, -0.6573, -0.3856]],
       grad_fn=<SelectBackward0>)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
id2label = {0: "Justice Breyer", 1: "Justice Ginsburg", 2: "Justice Kennedy", 3: "Justice O'Connor",
            4: "Justice Rehnquist", 5: "Justice Scalia",6: "Justice Souter", 7: "Justice Stevens", 8: "Justice Thomas"}
label2id = {"Justice Breyer": 0, "Justice Ginsburg": 1, "Justice Kennedy": 2, "Justice O'Connor":3,
            "Justice Rehnquist":4, "Justice Scalia":5, "Justice Souter": 6, "Justice Stevens":7, "Justice Thomas":8}

In [ ]:
# full code
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch.nn as nn
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from datasets import load_dataset, Dataset
import pandas as pd

###### Data Loading ###################
court = load_dataset('raminass/opinions_1994_2005', use_auth_token=True)
df = pd.DataFrame(court['train'])
court = Dataset.from_pandas(df[['label','text']])
# court = court.remove_columns('__index_level_0__')
court = court.class_encode_column("label")
court = court.train_test_split(0.2, stratify_by_column='label')

###### Model instance ###################
arch = 'legal-bert-small-uncased'
repo = "SCOTUS-1994-2005"
tokenizer = AutoTokenizer.from_pretrained(f"nlpaueb/{arch}")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_court = court.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(
    f"nlpaueb/{arch}", num_labels=9, id2label=id2label, label2id=label2id,
    # hidden_dropout_prob = 0.4, attention_probs_dropout_prob = 0.2
)

# model.classifier = nn.Sequential(
#             nn.Linear(512, 100),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(100, 50),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(50, 9),
#         )

# for param in model.bert.parameters():
#     param.requires_grad = False


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir=f"{repo}",
    logging_dir=f"{repo}/runs/fine-tune-with-cite",
    learning_rate=2e-5, #as in bert paper
    per_device_train_batch_size=16, # in legal-bert 256 however no memory here
    per_device_eval_batch_size=16, # in legal-bert 256 however no memory here
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_court["train"],
    eval_dataset=tokenized_court["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.push_to_hub()

  0%|          | 0/1 [00:00<?, ?it/s]

Stringifying the column:   0%|          | 0/13805 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/13805 [00:00<?, ? examples/s]

Map:   0%|          | 0/11044 [00:00<?, ? examples/s]

Map:   0%|          | 0/2761 [00:00<?, ? examples/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-small-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

RuntimeError: ignored